In [1]:
import sys
sys.path.insert(0, '../../')
from src.questions_construction.main import PLAN_LENGTHS, QUESTION_CATEGORIES
from src.questions_construction.domains import DOMAIN_NAMES

from src.analysis.model_performances import *
from src.common import *
import random

In [2]:
def gather_data_iterator():
    for substitutions in [WITHOUT_RANDOM_SUB]:
        for ramifications in [WITHOUT_RAMIFICATIONS]:
            for model_name in ['gpt-4o', 'gemini']:
                for prompt_type in ['few_shot_1']:
                    yield substitutions, ramifications, model_name, prompt_type

In [3]:
questions_dir = f'{DATA_PATH}/questions_m1'
ids_file_name = 'dataset_ids.test.pruned'

selected_ids = open_jsonl(f'{DATA_PATH}/{ids_file_name}.jsonl')

questions_by_id = gather_questions(questions_dir, selected_ids)
data_all, missing_data = gather_data(questions_by_id, selected_ids=selected_ids, iterator=gather_data_iterator)
save_main_dir = f'{STATISTICS_PATH}.{ids_file_name}'
sanity_checks(questions_by_id, data_all)

questions gathered


100%|██████████| 260/260 [00:12<00:00, 20.35it/s] 

data is gathered
checks passed


True

In [12]:
def prepare_dict(data):
    for d in data:
        d['model_parsed_response'] = find_text_within_brackets(d['response'])
        d['true_answer'] = d.pop('answer')
        for k in ['question', 'true_answer', 'model_parsed_response']:
            t_tmp = d.pop(k)
            d[k] = t_tmp
        
    return data

answer_type = FREE_ANSWER_TYPE
ramifications = WITHOUT_RAMIFICATIONS
substitutions = WITHOUT_RANDOM_SUB 
prompt_type = 'few_shot_1'
plan_length = 19
domain = ALL_DOMAINS_KEY

NUM_SAMPLES = 20

gemini_by_categories = {}
gpt_by_categories = {}
for question_category in QUESTION_CATEGORIES:
    data_gpt = filter_multi_selector(data_all, plan_length, question_category, ramifications, 'gpt-4o', prompt_type, domain,
                              answer_type, substitutions)
    data_gemini = filter_multi_selector(data_all, plan_length, question_category, ramifications, 'gemini', prompt_type, domain,
                              answer_type, substitutions)
    common_ids = list(set([d[OUT_OBJ_ID] for d in data_gpt]) & set([d[OUT_OBJ_ID] for d in data_gemini]))
    assert len(common_ids) > NUM_SAMPLES 
    
    
    random.shuffle(common_ids)
    common_ids = common_ids[:NUM_SAMPLES]
    
    gemini_by_categories[question_category] = prepare_dict([d for d in data_gemini if d[OUT_OBJ_ID] in common_ids])
    gpt_by_categories[question_category] =  prepare_dict([d for d in data_gpt if d[OUT_OBJ_ID] in common_ids])

In [13]:
# with open('human_eval_data.gemini.json', 'w') as f:
#     json.dump(gemini_by_categories, f)
# 
# with open('human_eval_data.gpt.json', 'w') as f:
#     json.dump(gpt_by_categories, f)
#     